<a href="https://colab.research.google.com/github/mariyajoseph2002/ml/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Embedding

In [3]:
from google.colab import files
uploaded = files.upload()

Saving train.txt to train.txt


In [4]:
data=pd.read_csv("train.txt",sep=";")
data.columns=["Text","Emotion"]
data.head(5)

,Text,Emotion
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     15999 non-null  object
 1   Emotion  15999 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


In [6]:
data.describe()

,Text,Emotion
count,15999,15999
unique,15968,6
top,i cant escape the tears of sadness and just tr...,joy
freq,2,5362


In [7]:
texts=data["Text"].tolist()
labels=data["Emotion"].tolist()

In [8]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)

In [9]:
sequences=tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences,maxlen = max_length)

In [10]:
le=LabelEncoder()
labels=le.fit_transform(labels)

In [11]:
one_hot_labels=keras.utils.to_categorical(labels)

In [12]:
x_train,x_test,y_train,y_test=train_test_split(padded_sequences,one_hot_labels,test_size=0.2)

In [13]:
model=Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=128,input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]),activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.3830 - loss: 1.5172 - val_accuracy: 0.7287 - val_loss: 0.8080
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.8753 - loss: 0.3936 - val_accuracy: 0.8153 - val_loss: 0.5626
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9850 - loss: 0.0630 - val_accuracy: 0.8128 - val_loss: 0.6264
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9966 - loss: 0.0198 - val_accuracy: 0.8222 - val_loss: 0.6476
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9973 - loss: 0.0159 - val_accuracy: 0.8153 - val_loss: 0.7208
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.9980 - loss: 0.0093 - val_accuracy: 0.8147 - val_loss: 0.7061
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.9984 - loss: 0.0086 - val_accuracy: 0.8163 - val_loss: 0.7252
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9981 - loss: 0.0095 - val_accurac

In [25]:
input_text="i feel intimidated by his actions"
input_sequence=tokenizer.texts_to_sequences([input_text])
padded_input_sequence=pad_sequences(input_sequence,maxlen=max_length)
prediction=model.predict(padded_input_sequence)
predicted_label=le.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
['fear']
